Churn prediction and Customer segmentation

In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import resample


from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from scipy.stats import pearsonr, spearmanr

from matplotlib import pyplot as plt
from sklearn.utils import resample


from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score


In [4]:
data = pd.read_csv('Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [5]:
data = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1) 
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9997,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10002 non-null  int64  
 1   Geography        10001 non-null  object 
 2   Gender           10002 non-null  object 
 3   Age              10001 non-null  float64
 4   Tenure           10002 non-null  int64  
 5   Balance          10002 non-null  float64
 6   NumOfProducts    10002 non-null  int64  
 7   HasCrCard        10001 non-null  float64
 8   IsActiveMember   10001 non-null  float64
 9   EstimatedSalary  10002 non-null  float64
 10  Exited           10002 non-null  int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 859.7+ KB


In [7]:
data.shape

(10002, 11)

In [8]:
data.isnull().sum()

CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
data[data.duplicated()]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9999,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10001,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [10]:
data.dropna(inplace=True)
data.drop_duplicates(keep='first', inplace = True)

In [11]:
data.shape

(9996, 11)

In [12]:
print(spearmanr(data['Gender'], data['Exited']).correlation)
print(spearmanr(data['Geography'], data['Exited']).correlation)
print(spearmanr(data['Age'], data['Exited']).correlation)
print(spearmanr(data['NumOfProducts'], data['Exited']).correlation)
print(spearmanr(data['IsActiveMember'], data['Exited']).correlation)
print(spearmanr(data['Tenure'], data['Exited']).correlation)


-0.10645458137656881
0.05302847198378358
0.32410806470608133
-0.12531397832251814
-0.15596927099238198
-0.014068357358581675


In [13]:
data['Gender'].value_counts()

Male      5454
Female    4542
Name: Gender, dtype: int64

In [14]:
data['Geography'].value_counts()

France     5011
Germany    2509
Spain      2476
Name: Geography, dtype: int64

In [15]:
data['Tenure'].value_counts()

2     1046
1     1035
7     1027
8     1025
5     1012
3     1009
4      988
9      984
6      967
10     490
0      413
Name: Tenure, dtype: int64

In [16]:
data['NumOfProducts'].value_counts()

1    5082
2    4588
3     266
4      60
Name: NumOfProducts, dtype: int64

In [17]:
aa = pd.crosstab(data.Geography, data.Exited)
for i in aa.index:
    s = sum(aa.loc[i].values)
    print(f"The probability of residents of {i} churning is approximately", round(aa.loc[i].values[1]/s, 3))


The probability of residents of France churning is approximately 0.162
The probability of residents of Germany churning is approximately 0.324
The probability of residents of Spain churning is approximately 0.167


In [18]:
aa = pd.crosstab(data.Gender, data.Exited)
for i in aa.index:
    s = sum(aa.loc[i].values)
    print(f"The probability of {i.lower()} customer churning is approximately", round(aa.loc[i].values[1]/s, 3))


The probability of female customer churning is approximately 0.251
The probability of male customer churning is approximately 0.165


In [19]:
aa = pd.crosstab(data.IsActiveMember, data.Exited)
for i in aa.index:
    s = sum(aa.loc[i].values)
    if i == 0:
        j = 'non-active'
    else:
        j = 'active'
    print(f"The probability of {j} users to churn is approximately", round(aa.loc[i].values[1]/s, 3))


The probability of non-active users to churn is approximately 0.269
The probability of active users to churn is approximately 0.143


In [20]:
aa = pd.crosstab(data.NumOfProducts, data.Exited)
aa

Exited,0,1
NumOfProducts,,
1,3673,1409
2,4240,348
3,46,220
4,0,60


In [21]:
aa = pd.crosstab(data.NumOfProducts, data.Exited)
for i in aa.index:
    s = sum(aa.loc[i].values)
    print(f"The probability of customers with {i} products of churning is approximately", round(aa.loc[i].values[1]/s, 3))



The probability of customers with 1 products of churning is approximately 0.277
The probability of customers with 2 products of churning is approximately 0.076
The probability of customers with 3 products of churning is approximately 0.827
The probability of customers with 4 products of churning is approximately 1.0


In [22]:
aa = pd.crosstab(data.HasCrCard, data.Exited)
for i in aa.index:
    s = sum(aa.loc[i].values)
    if i == 0:
        j = 'without'
    else:
        j = 'with'
    print(f"The probability of customers {j} credit card of churning is approximately", round(aa.loc[i].values[1]/s, 3))

The probability of customers without credit card of churning is approximately 0.208
The probability of customers with credit card of churning is approximately 0.202


In [23]:
data.Geography = data.Geography.map({'France': 0, 'Germany': 1, 'Spain': 2})
data.Gender = data.Gender.map({'Female': 0, 'Male': 1})

In [24]:
y = data.Exited
data.drop(columns = ['Exited'], inplace = True)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.3)

In [26]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
list_of_models = {
    'logistic_regression' : LogisticRegression(random_state = 5, max_iter = 1000),
    'decision_tree' : DecisionTreeClassifier(max_depth = 5, random_state = 42,criterion='entropy'),
    'kneighbirs' : KNeighborsClassifier(n_neighbors=3),
    'Gaussian' : GaussianNB(),
    "svm" : svm.SVC(kernel='rbf') 
}

In [28]:
accuracy_s = []
f1_train_scores = [] 
f1_test_scores = [] 
recall_test_scores = []

model_names = list_of_models.keys()

for model in model_names:
    
    list_of_models[model].fit(X_train, y_train)
    y_train_pred = list_of_models[model].predict(X_train)
    

    f1_train = f1_score(y_train, y_train_pred, average='macro')

    y_test_pred = list_of_models[model].predict(X_test)
    
    
    f1_test = f1_score(y_test, y_test_pred, average='macro')

    recall_test_score = recall_score(y_test, y_test_pred, average='macro')
    
    accuracy_s.append(accuracy_score(y_test, y_test_pred))
    f1_train_scores.append(f1_train)
    f1_test_scores.append(f1_test)
    recall_test_scores.append(recall_test_score)


d = {
    'accuracy_s': accuracy_s,
    'f1_train_scores':f1_train_scores,
    'f1 test_scores': f1_test_scores,
    'Recall test scores': recall_test_scores
}
df = pd.DataFrame(d)
df.index = ['logistic_regression','decision_tree' ,'kneighbors' ,'Gaussian','svm']
print(df)

 


                     accuracy_s  f1_train_scores  f1 test_scores  \
logistic_regression    0.807936         0.587537        0.589786   
decision_tree          0.851951         0.736999        0.722811   
kneighbors             0.816272         0.825201        0.679773   
Gaussian               0.826942         0.638987        0.629996   
svm                    0.846616         0.733313        0.695414   

                     Recall test scores  
logistic_regression            0.579141  
decision_tree                  0.689779  
kneighbors                     0.661891  
Gaussian                       0.608161  
svm                            0.661405  


In [29]:
import numpy
import tensorflow

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a simple neural network model
model = Sequential([Dense(10, activation='sigmoid'), Dense(1, activation='sigmoid')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3)

y_train_pred = model.predict(X_train)

print(accuracy_score(y_test, y_test_pred))


Epoch 1/10
154/154 [==============================] - 1s 3ms/step - loss: 0.5641 - accuracy: 0.7788 - val_loss: 0.5093 - val_accuracy: 0.8000
Epoch 2/10
154/154 [==============================] - 0s 2ms/step - loss: 0.4952 - accuracy: 0.7944 - val_loss: 0.4786 - val_accuracy: 0.8000
Epoch 3/10
154/154 [==============================] - 0s 2ms/step - loss: 0.4728 - accuracy: 0.7944 - val_loss: 0.4637 - val_accuracy: 0.8000
Epoch 4/10
154/154 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.7946 - val_loss: 0.4544 - val_accuracy: 0.7995
Epoch 5/10
154/154 [==============================] - 0s 2ms/step - loss: 0.4496 - accuracy: 0.7974 - val_loss: 0.4479 - val_accuracy: 0.8024
Epoch 6/10
154/154 [==============================] - 0s 3ms/step - loss: 0.4426 - accuracy: 0.8009 - val_loss: 0.4433 - val_accuracy: 0.8105
Epoch 7/10
154/154 [==============================] - 0s 2ms/step - loss: 0.4376 - accuracy: 0.8040 - val_loss: 0.4402 - val_accuracy: 0.8090
Epoch 

In [46]:

# Create a simple neural network model
model = Sequential([Dense(100, activation='relu'), Dense(10, activation='relu'),Dense(1, activation='relu')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(X_train)

Epoch 1/10
175/175 [==============================] - 1s 4ms/step - loss: 1.5595 - accuracy: 0.8031 - val_loss: 0.7237 - val_accuracy: 0.8221
Epoch 2/10
175/175 [==============================] - 0s 3ms/step - loss: 0.5004 - accuracy: 0.8190 - val_loss: 0.5573 - val_accuracy: 0.8443
Epoch 3/10
175/175 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.8321 - val_loss: 0.4759 - val_accuracy: 0.8507
Epoch 4/10
175/175 [==============================] - 0s 2ms/step - loss: 0.4649 - accuracy: 0.8356 - val_loss: 0.4602 - val_accuracy: 0.8457
Epoch 5/10
175/175 [==============================] - 0s 2ms/step - loss: 0.4234 - accuracy: 0.8446 - val_loss: 0.5046 - val_accuracy: 0.8600
Epoch 6/10
175/175 [==============================] - 0s 2ms/step - loss: 0.4216 - accuracy: 0.8533 - val_loss: 0.5272 - val_accuracy: 0.8593
Epoch 7/10
175/175 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8537 - val_loss: 0.4954 - val_accuracy: 0.8643
Epoch 

In [54]:

# Create a simple neural network model
model = Sequential([Dense(150, activation='relu'), Dense(100, activation='relu'),Dense(10, activation='relu'),Dense(1, activation='relu')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(X_train)

Epoch 1/10
175/175 [==============================] - 3s 5ms/step - loss: 0.4696 - accuracy: 0.8183 - val_loss: 0.4060 - val_accuracy: 0.8521
Epoch 2/10
175/175 [==============================] - 1s 5ms/step - loss: 0.4551 - accuracy: 0.8356 - val_loss: 0.4019 - val_accuracy: 0.8557
Epoch 3/10
175/175 [==============================] - 1s 4ms/step - loss: 0.3771 - accuracy: 0.8485 - val_loss: 0.3940 - val_accuracy: 0.8529
Epoch 4/10
175/175 [==============================] - 1s 3ms/step - loss: 0.3531 - accuracy: 0.8503 - val_loss: 0.4105 - val_accuracy: 0.8457
Epoch 5/10
175/175 [==============================] - 1s 4ms/step - loss: 0.3452 - accuracy: 0.8556 - val_loss: 0.4258 - val_accuracy: 0.8586
Epoch 6/10
175/175 [==============================] - 1s 4ms/step - loss: 0.3330 - accuracy: 0.8583 - val_loss: 0.4057 - val_accuracy: 0.8593
Epoch 7/10
175/175 [==============================] - 1s 4ms/step - loss: 0.3339 - accuracy: 0.8583 - val_loss: 0.3806 - val_accuracy: 0.8607
Epoch 